<a href="https://colab.research.google.com/github/snbigft/api_rest_lotto/blob/main/Api_web_lotto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn psycopg2-binary nest-asyncio pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00


In [ ]:
import json
import psycopg2
import nest_asyncio
import uvicorn
from fastapi import FastAPI, Query
from pyngrok import ngrok

# Aplicar nest_asyncio para evitar conflictos de bucle de eventos en Colab
nest_asyncio.apply()

# Configurar ngrok con tu authtoken
ngrok.set_auth_token("2tfZlVTSTp5jhPzAVI8iNoKRIwq_6NKUQdERg7XoqNdkJsuNA")

# Configuración de la base de datos
DB_HOST = "ep-snowy-dawn-a8omlpke-pooler.eastus2.azure.neon.tech"
DB_NAME = "neondb"
DB_USER = "neondb_owner"
DB_PASSWORD = "npg_BcpnId62FYhS"

# Función para conectar a PostgreSQL
def get_db_connection():
    return psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        sslmode="require"
    )

# Crear la API con FastAPI
app = FastAPI()

# Opciones válidas para los endpoints
RUOTA_OPTIONS = ["Bari", "Cagliari", "Firenze", "Genova", "Milano", "Napoli", "Palermo", "Roma", "Torino", "Venezia", "Nazionale"]
TIPO_OPTIONS = ["top 10 ritardatari", "top 10 frequenti", "ambo piú frequente", "top 10 ambi"]


# Definir las etiquetas para cada endpoint y tipo de consulta
labels_mapping = {
    "/absolute_statistics": {
        "top 10 ritardatari": ["ultima_estrazione", "numero", "ruota"],
        "top 10 frequenti": ["numero_estrazioni", "numero", "ruota"],
        "ambo piú frequente": ["numero_estrazioni", "numero_1", "numero_2", "ruota"],
        "top 10 ambi": ["numero_estrazioni", "numero_1", "numero_2", "ruota"]
    },
    "/number_info": ["numero_estrazioni", "numero", "ruota", "ultima_estrazione"],
    "/ambo_info": ["numero_estrazioni", "numero_1", "numero_2", "ruota", "ultima_estrazione"]
}

# Endpoint existente para customers
@app.get("/customers")
def get_customer_data(query: str = Query(..., description="Valor a buscar en customer_data")):
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT customer_data FROM CUSTOMERS WHERE customer_data::text ILIKE %s", (f"%{query}%",))
        rows = cursor.fetchall()
        result = [row[0] for row in rows]
        cursor.close()
        conn.close()
        return {"results": result}
    except Exception as e:
        return {"error": str(e)}

# Endpoint para absolute_statistics
@app.get("/absolute_statistics")
def absolute_statistics(ruota: str = Query(None, enum=RUOTA_OPTIONS), tipo: str = Query(..., enum=TIPO_OPTIONS)):
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        if tipo == 'top 10 ritardatari':
            if ruota:
                query = f"""
                    SELECT MAX(date_sort) ultima_estrazione, numero, ruota
                    FROM lotto_it
                    WHERE ruota='{ruota}'
                    GROUP BY numero, ruota
                    ORDER BY 1 ASC
                    LIMIT 10;
                """
            else:
                query = f"""
                    SELECT MAX(date_sort) ultima_estrazione, numero,ruota
                    FROM lotto_it
                    GROUP BY numero, ruota
                    ORDER BY 1 ASC
                    LIMIT 10;
                """
        elif tipo == 'top 10 frequenti':
            if ruota:
                query = f"""
                    SELECT COUNT(*) numero_estrazioni , numero , ruota
                    FROM lotto_it
                    WHERE ruota='{ruota}'
                    GROUP BY ruota, numero
                    ORDER BY 1 DESC
                    LIMIT 10;
                """
            else:
                query = f"""
                    SELECT COUNT(*) numero_estrazioni , numero , ruota
                    FROM lotto_it
                    GROUP BY ruota, numero
                    ORDER BY 1 DESC
                    LIMIT 10;
                """
        elif tipo == 'ambo piú frequente':
            if ruota:
                query = f"""
                    SELECT ocurrencias numero_estrazioni , numero_1 , numero_2  , ruota
                    FROM ambos_matrix
                    WHERE ocurrencias = (
                        SELECT MAX(ocurrencias)
                        FROM ambos_matrix
                        WHERE ocurrencias != 0 AND ruota='{ruota}'
                    );
                """
            else:
                query = f"""
                    SELECT MAX(veces) numero_estrazioni, numero_1, numero_2 , 'tutte' ruota
                    FROM (
                        SELECT SUM(ocurrencias) veces, numero_1, numero_2
                        FROM ambos_matrix
                        WHERE ocurrencias != 0
                        GROUP BY numero_1, numero_2
                    ) subquery
                    GROUP BY numero_1, numero_2
                    ORDER BY 1 DESC
                    LIMIT 10;
                """

        elif tipo == 'top 10 ambi':
            if ruota:
                query = f"""
                    WITH sum_veces AS (
                        SELECT SUM(ocurrencias) veces , numero_1, numero_2, ruota
                        FROM ambos_matrix
                        WHERE ocurrencias != 0
                        GROUP BY numero_1, numero_2, ruota
                    ),
                    RankedNumbers AS (
                        SELECT veces, ruota, numero_1, numero_2,
                               RANK() OVER (PARTITION BY ruota ORDER BY veces DESC) AS rank
                        FROM sum_veces
                    )
                    SELECT veces numero_estrazioni,  numero_1, numero_2 , ruota
                    FROM RankedNumbers
                    WHERE rank <= 10 AND ruota='{ruota}'
                    ORDER BY ruota, veces DESC;
                """
            else:
                query = f"""
                    WITH sum_veces AS (
                        SELECT SUM(ocurrencias) veces , numero_1, numero_2, ruota
                        FROM ambos_matrix
                        WHERE ocurrencias != 0
                        GROUP BY numero_1, numero_2, ruota
                    ),
                    RankedNumbers AS (
                        SELECT veces, ruota, numero_1, numero_2,
                               RANK() OVER (PARTITION BY ruota ORDER BY veces DESC) AS rank
                        FROM sum_veces
                    )
                    SELECT veces numero_estrazioni, numero_1, numero_2 , ruota
                    FROM RankedNumbers
                    WHERE rank <= 10
                    ORDER BY ruota, veces DESC;
                """
        cursor.execute(query)
        results = cursor.fetchall()
        endpoint = "/absolute_statistics"
        selected_labels = labels_mapping[endpoint].get(tipo, [])

        # Formatear la respuesta con las etiquetas dinámicas
        formatted_results = [
            dict(zip(selected_labels, row))
            for row in results
        ]



        cursor.close()
        conn.close()
        return {"results": formatted_results}

    except Exception as e:
        return {"error": str(e)}

# Endpoint para number_info
@app.get("/number_info")
def number_info(ruota: str = Query(None, enum=RUOTA_OPTIONS), numero: int = Query(..., ge=1, le=90)):
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        if ruota:
            query = f"""
                WITH max_count_per_ruota AS (
                    SELECT COUNT(*) numero_estrazioni, ruota, numero
                    FROM lotto_it
                    WHERE ruota='{ruota}' AND numero={numero}
                    GROUP BY ruota, numero),
                last_time_per_ruota AS (
                    SELECT MAX(date_sort) ultima_estrazione, ruota, numero
                    FROM lotto_it
                    WHERE ruota='{ruota}' AND numero={numero}
                    GROUP BY ruota, numero)
                SELECT max_c.numero_estrazioni, max_c.numero , max_c.ruota ,last_c.ultima_estrazione
                FROM max_count_per_ruota max_c
                JOIN last_time_per_ruota last_c
                ON max_c.ruota = last_c.ruota AND max_c.numero = last_c.numero;
            """
        else:
            query = f"""
                WITH max_count_per_ruota AS (
                    SELECT COUNT(*) numero_estrazioni, ruota, numero
                    FROM lotto_it
                    WHERE numero={numero}
                    GROUP BY ruota, numero
                    ORDER BY 1 DESC LIMIT 10),
                last_time_per_ruota AS (
                    SELECT MAX(date_sort) ultima_estrazione, ruota, numero
                    FROM lotto_it
                    WHERE numero={numero}
                    GROUP BY ruota, numero
                    ORDER BY 1 DESC LIMIT 10)
                SELECT max_c.numero_estrazioni, max_c.numero , max_c.ruota, last_c.ultima_estrazione
                FROM max_count_per_ruota max_c
                JOIN last_time_per_ruota last_c
                ON max_c.ruota = last_c.ruota AND max_c.numero = last_c.numero;
            """

        cursor.execute(query)
        results = cursor.fetchall()
        endpoint = "/number_info"
        selected_labels = labels_mapping.get(endpoint, [])

        # Formatear la respuesta con las etiquetas dinámicas
        formatted_results = [
            dict(zip(selected_labels, row))
            for row in results
        ]

        cursor.close()
        conn.close()
        return {"results": formatted_results}

    except Exception as e:
        return {"error": str(e)}


# Endpoint para ambo_info
@app.get("/ambo_info")
def number_info(ruota: str = Query(None, enum=RUOTA_OPTIONS), numero_1: int = Query(..., ge=1, le=90), numero_2: int = Query(..., ge=1, le=90)):
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        if ruota:
            query = f"""
                WITH first_number as (
                  SELECT numero numero_1, ruota , date_sort
                  FROM lotto_it
                  WHERE ruota='{ruota}' AND numero={numero_1}),
                second_number as (SELECT numero numero_2, ruota , date_sort
                  FROM lotto_it
                  WHERE ruota='{ruota}' AND numero={numero_2}),
                max_ambo_frecuency as (
                  SELECT count(*) frecuencia_ambo, fn.numero_1 , sn.numero_2 , fn.ruota
                  FROM first_number fn
                  JOIN second_number sn ON fn.date_sort=sn.date_sort AND fn.ruota=sn.ruota
                  GROUP BY fn.numero_1 , sn.numero_2 , fn.ruota)
                SELECT  maf.frecuencia_ambo numero_estrazioni, fn.numero_1 , sn.numero_2 , fn.ruota ,fn.date_sort ultima_estrazione
                FROM first_number fn
                JOIN second_number sn ON fn.date_sort=sn.date_sort AND fn.ruota=sn.ruota
                JOIN max_ambo_frecuency maf ON  maf.numero_1=fn.numero_1 AND maf.numero_2=sn.numero_2 AND maf.ruota=fn.ruota
                ORDER BY fn.date_sort DESC
                LIMIT 10;
            """
        else:
            query = f"""
                 WITH first_number as (
                  SELECT numero numero_1, ruota , date_sort
                  FROM lotto_it
                  WHERE numero={numero_1}),
                second_number as (
                  SELECT  numero numero_2, ruota , date_sort
                  FROM lotto_it
                  WHERE numero={numero_2}),
                max_ambo_frecuency as (
                  SELECT count(*) frecuencia_ambo, max(fn.date_sort) ultima_extraccion, fn.numero_1 , sn.numero_2 , fn.ruota
                  FROM first_number fn
                  JOIN second_number sn ON fn.date_sort=sn.date_sort AND fn.ruota=sn.ruota
                  GROUP BY fn.numero_1 , sn.numero_2 , fn.ruota)
                SELECT DISTINCT maf.frecuencia_ambo numero_estrazioni, fn.numero_1 , sn.numero_2 , fn.ruota, maf.ultima_extraccion
                FROM first_number fn
                JOIN second_number sn ON fn.date_sort=sn.date_sort AND fn.ruota=sn.ruota
                JOIN max_ambo_frecuency maf ON  maf.numero_1=fn.numero_1 AND maf.numero_2=sn.numero_2 AND maf.ruota=fn.ruota
                ORDER BY maf.frecuencia_ambo desc LIMIT 10;
            """

        cursor.execute(query)
        results = cursor.fetchall()
        endpoint = "/ambo_info"
        selected_labels = labels_mapping.get(endpoint, [])

        # Formatear la respuesta con las etiquetas dinámicas
        formatted_results = [
            dict(zip(selected_labels, row))
            for row in results
        ]
        cursor.close()
        conn.close()
        return {"results": formatted_results}

    except Exception as e:
        return {"error": str(e)}


# Exponer la API con ngrok en Google Colab
public_url = ngrok.connect(8000).public_url
print(f"🔥 API en ejecución en: {public_url}/docs")

# Iniciar el servidor uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)

🔥 API en ejecución en: https://0865-35-204-143-49.ngrok-free.app/docs


INFO:     Started server process [948]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     195.235.159.28:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     195.235.159.28:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     195.235.159.28:0 - "GET /absolute_statistics?tipo=top%2010%20ritardatari HTTP/1.1" 200 OK
INFO:     195.235.159.28:0 - "GET /absolute_statistics?tipo=ambo%20pi%C3%BA%20frequente HTTP/1.1" 200 OK
INFO:     195.235.159.28:0 - "GET /absolute_statistics?tipo=top%2010%20ambi HTTP/1.1" 200 OK


import json
import psycopg2
import nest_asyncio
import uvicorn
from fastapi import FastAPI, Query
from pyngrok import ngrok

# Aplicar nest_asyncio para evitar conflictos de bucle de eventos en Colab
nest_asyncio.apply()

# Configurar ngrok con tu authtoken
ngrok.set_auth_token("2tfZlVTSTp5jhPzAVI8iNoKRIwq_6NKUQdERg7XoqNdkJsuNA")

# Configuración de la base de datos
DB_HOST = "ep-snowy-dawn-a8omlpke-pooler.eastus2.azure.neon.tech"
DB_NAME = "neondb"
DB_USER = "neondb_owner"
DB_PASSWORD = "npg_BcpnId62FYhS"

# Función para conectar a PostgreSQL
def get_db_connection():
    return psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        sslmode="require"
    )

# Crear la API con FastAPI
app = FastAPI()

# Opciones válidas para los endpoints
RUOTA_OPTIONS = ["Bari", "Cagliari", "Firenze", "Genova", "Milano", "Napoli", "Palermo", "Roma", "Torino", "Venezia", "Nazionale"]
TIPO_OPTIONS = ["top 10 ritardatari", "top 10 frequenti", "ambo piú frequente", "top 10 ambi"]


# Definir las etiquetas para cada endpoint y tipo de consulta
labels_mapping = {
    "/absolute_statistics": {
        "top 10 ritardatari": ["ultima_estrazione", "numero", "ruota"],
        "top 10 frequenti": ["numero_estrazioni", "numero", "ruota"],
        "ambo piú frequente": ["numero_estrazioni", "numero_1", "numero_2", "ruota"],
        "top 10 ambi": ["numero_estrazioni", "numero_1", "numero_2", "ruota"]
    },
    "/number_info": ["numero_estrazioni", "numero", "ruota", "ultima_estrazione"],
    "/ambo_info": ["numero_estrazioni", "numero_1", "numero_2", "ruota", "ultima_estrazione"]
}

# Endpoint existente para customers
@app.get("/customers")
def get_customer_data(query: str = Query(..., description="Valor a buscar en customer_data")):
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT customer_data FROM CUSTOMERS WHERE customer_data::text ILIKE %s", (f"%{query}%",))
        rows = cursor.fetchall()
        result = [row[0] for row in rows]
        cursor.close()
        conn.close()
        return {"results": result}
    except Exception as e:
        return {"error": str(e)}

# Endpoint para absolute_statistics
@app.get("/absolute_statistics")
def absolute_statistics(ruota: str = Query(None, enum=RUOTA_OPTIONS), tipo: str = Query(..., enum=TIPO_OPTIONS)):
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        if tipo == 'top 10 ritardatari':
            if ruota:
                query = f"""
                    SELECT MAX(date_sort) ultima_estrazione, numero, ruota
                    FROM lotto_it
                    WHERE ruota='{ruota}'
                    GROUP BY numero, ruota
                    ORDER BY 1 ASC
                    LIMIT 10;
                """
            else:
                query = f"""
                    SELECT MAX(date_sort) ultima_estrazione, numero,ruota,
                    FROM lotto_it
                    GROUP BY numero, ruota
                    ORDER BY 1 ASC
                    LIMIT 10;
                """
        elif tipo == 'top 10 frequenti':
            if ruota:
                query = f"""
                    SELECT COUNT(*) numero_estrazioni , numero , ruota
                    FROM lotto_it
                    WHERE ruota='{ruota}'
                    GROUP BY ruota, numero
                    ORDER BY 1 DESC
                    LIMIT 10;
                """
            else:
                query = f"""
                    SELECT COUNT(*) numero_estrazioni , numero , ruota
                    FROM lotto_it
                    GROUP BY ruota, numero
                    ORDER BY 1 DESC
                    LIMIT 10;
                """
        elif tipo == 'ambo piú frequente':
            if ruota:
                query = f"""
                    SELECT ocurrencias numero_estrazioni , numero_1 , numero_2  , ruota
                    FROM ambos_matrix
                    WHERE ocurrencias = (
                        SELECT MAX(ocurrencias)
                        FROM ambos_matrix
                        WHERE ocurrencias != 0 AND ruota='{ruota}'
                    );
                """
            else:
                query = f"""
                    SELECT MAX(veces) numero_estrazioni, numero_1, numero_2 , 'tutte' ruota
                    FROM (
                        SELECT SUM(ocurrencias) veces, numero_1, numero_2
                        FROM ambos_matrix
                        WHERE ocurrencias != 0
                        GROUP BY numero_1, numero_2
                    ) subquery
                    GROUP BY numero_1, numero_2
                    ORDER BY 1 DESC
                    LIMIT 10;
                """

        elif tipo == 'top 10 ambi':
            if ruota:
                query = f"""
                    WITH sum_veces AS (
                        SELECT SUM(ocurrencias) numero_estrazioni , numero_1, numero_2, ruota
                        FROM ambos_matrix
                        WHERE ocurrencias != 0
                        GROUP BY numero_1, numero_2, ruota
                    ),
                    RankedNumbers AS (
                        SELECT veces, ruota, numero_1, numero_2,
                               RANK() OVER (PARTITION BY ruota ORDER BY veces DESC) AS rank
                        FROM sum_veces
                    )
                    SELECT veces, ruota, numero_1, numero_2
                    FROM RankedNumbers
                    WHERE rank <= 10 AND ruota='{ruota}'
                    ORDER BY ruota, veces DESC;
                """
            else:
                query = f"""
                    WITH sum_veces AS (
                        SELECT SUM(ocurrencias) numero_estrazioni , numero_1, numero_2, ruota
                        FROM ambos_matrix
                        WHERE ocurrencias != 0
                        GROUP BY numero_1, numero_2, ruota
                    ),
                    RankedNumbers AS (
                        SELECT veces, ruota, numero_1, numero_2,
                               RANK() OVER (PARTITION BY ruota ORDER BY veces DESC) AS rank
                        FROM sum_veces
                    )
                    SELECT veces, numero_1, numero_2 , ruota
                    FROM RankedNumbers
                    WHERE rank <= 10
                    ORDER BY ruota, veces DESC;
                """
        cursor.execute(query)
        results = cursor.fetchall()
        if endpoint == "/absolute_statistics":
          selected_labels = labels_mapping[endpoint].get(tipo, [])
        else:
          selected_labels = labels_mapping.get(endpoint, [])

        # Formatear la respuesta con las etiquetas dinámicas
        formatted_results = [
            dict(zip(selected_labels, row))
            for row in results
        ]



        cursor.close()
        conn.close()
        return {"results": formatted_results}

    except Exception as e:
        return {"error": str(e)}

# Endpoint para number_info
@app.get("/number_info")
def number_info(ruota: str = Query(None, enum=RUOTA_OPTIONS), numero: int = Query(..., ge=1, le=90)):
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        if ruota:
            query = f"""
                WITH max_count_per_ruota AS (
                    SELECT COUNT(*) numero_estrazioni, ruota, numero
                    FROM lotto_it
                    WHERE ruota='{ruota}' AND numero={numero}
                    GROUP BY ruota, numero),
                last_time_per_ruota AS (
                    SELECT MAX(date_sort) ultima_estrazione, ruota, numero
                    FROM lotto_it
                    WHERE ruota='{ruota}' AND numero={numero}
                    GROUP BY ruota, numero)
                SELECT max_c.numero_estrazioni, max_c.numero , max_c.ruota ,last_c.ultima_estrazione
                FROM max_count_per_ruota max_c
                JOIN last_time_per_ruota last_c
                ON max_c.ruota = last_c.ruota AND max_c.numero = last_c.numero;
            """
        else:
            query = f"""
                WITH max_count_per_ruota AS (
                    SELECT COUNT(*) numero_estrazioni, ruota, numero
                    FROM lotto_it
                    WHERE numero={numero}
                    GROUP BY ruota, numero
                    ORDER BY 1 DESC LIMIT 10),
                last_time_per_ruota AS (
                    SELECT MAX(date_sort) ultima_estrazione, ruota, numero
                    FROM lotto_it
                    WHERE numero={numero}
                    GROUP BY ruota, numero
                    ORDER BY 1 DESC LIMIT 10)
                SELECT max_c.numero_estrazioni, max_c.numero , max_c.ruota, last_c.ultima_estrazione
                FROM max_count_per_ruota max_c
                JOIN last_time_per_ruota last_c
                ON max_c.ruota = last_c.ruota AND max_c.numero = last_c.numero;
            """

        cursor.execute(query)
        results = cursor.fetchall()
        cursor.close()
        conn.close()
        return {"results": results}

    except Exception as e:
        return {"error": str(e)}


# Endpoint para ambo_info
@app.get("/ambo_info")
def number_info(ruota: str = Query(None, enum=RUOTA_OPTIONS), numero_1: int = Query(..., ge=1, le=90), numero_2: int = Query(..., ge=1, le=90)):
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        if ruota:
            query = f"""
                WITH first_number as (
                  SELECT numero numero_1, ruota , date_sort
                  FROM lotto_it
                  WHERE ruota='{ruota}' AND numero={numero_1}),
                second_number as (SELECT numero numero_2, ruota , date_sort
                  FROM lotto_it
                  WHERE ruota='{ruota}' AND numero={numero_2}),
                max_ambo_frecuency as (
                  SELECT count(*) frecuencia_ambo, fn.numero_1 , sn.numero_2 , fn.ruota
                  FROM first_number fn
                  JOIN second_number sn ON fn.date_sort=sn.date_sort AND fn.ruota=sn.ruota
                  GROUP BY fn.numero_1 , sn.numero_2 , fn.ruota)
                SELECT  maf.frecuencia_ambo numero_estrazioni, fn.numero_1 , sn.numero_2 , fn.ruota ,fn.date_sort
                FROM first_number fn
                JOIN second_number sn ON fn.date_sort=sn.date_sort AND fn.ruota=sn.ruota
                JOIN max_ambo_frecuency maf ON  maf.numero_1=fn.numero_1 AND maf.numero_2=sn.numero_2 AND maf.ruota=fn.ruota
                ORDER BY fn.date_sort DESC
                LIMIT 10;
            """
        else:
            query = f"""
                 WITH first_number as (
                  SELECT numero numero_1, ruota , date_sort
                  FROM lotto_it
                  WHERE numero={numero_1}),
                second_number as (
                  SELECT  numero numero_2, ruota , date_sort
                  FROM lotto_it
                  WHERE numero={numero_2}),
                max_ambo_frecuency as (
                  SELECT count(*) frecuencia_ambo, max(fn.date_sort) ultima_extraccion, fn.numero_1 , sn.numero_2 , fn.ruota
                  FROM first_number fn
                  JOIN second_number sn ON fn.date_sort=sn.date_sort AND fn.ruota=sn.ruota
                  GROUP BY fn.numero_1 , sn.numero_2 , fn.ruota)
                SELECT DISTINCT maf.frecuencia_ambo, fn.numero_1 , sn.numero_2 , fn.ruota, maf.ultima_extraccion
                FROM first_number fn
                JOIN second_number sn ON fn.date_sort=sn.date_sort AND fn.ruota=sn.ruota
                JOIN max_ambo_frecuency maf ON  maf.numero_1=fn.numero_1 AND maf.numero_2=sn.numero_2 AND maf.ruota=fn.ruota
                ORDER BY maf.frecuencia_ambo desc LIMIT 10;
            """

        cursor.execute(query)
        results = cursor.fetchall()
        cursor.close()
        conn.close()
        return {"results": results}

    except Exception as e:
        return {"error": str(e)}


# Exponer la API con ngrok en Google Colab
public_url = ngrok.connect(8000).public_url
print(f"🔥 API en ejecución en: {public_url}/docs")

# Iniciar el servidor uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)